## Machine Learning - Final Project ##
---
---

In [1]:
# Imports
import os
import requests
import sqlglot
import ast
import re
import json
import pandas as pd
import tempfile

from requests.auth import HTTPBasicAuth
from docx import Document
from sqlglot import parse_one, ParseError
from dotenv import load_dotenv
from openai import OpenAI
from typing import List, Dict, Any, Tuple, Optional, Iterable
from requests.auth import HTTPBasicAuth
from pathlib import Path

In [2]:
# show the entire description column (no truncation)
pd.set_option("display.max_colwidth", None)

---
### Load Parameters ###

In [3]:
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

JIRA_EMAIL = os.getenv("JIRA_EMAIL")
JIRA_API_TOKEN = os.getenv("JIRA_API_TOKEN")
JIRA_BASE_URL = os.getenv("JIRA_BASE_URL")
JIRA_PROJECT_KEY = os.getenv("JIRA_PROJECT_KEY")

JIRA_FIELD_ASSIGNED_GROUP=os.getenv("JIRA_FIELD_ASSIGNED_GROUP", "")

JIRA_EPIC_NAME_FIELD = os.getenv("JIRA_EPIC_NAME_FIELD", "")
JIRA_EPIC_LINK_FIELD = os.getenv("JIRA_EPIC_LINK_FIELD", "")
TEAM_MANAGED_EPIC_PARENT = os.getenv("TEAM_MANAGED_EPIC_PARENT", "false").lower() == "true"
USE_EPIC_NAME = os.getenv("USE_EPIC_NAME", "false").lower() == "true"

---
### Read Text from .docx ###

In [4]:
def read_docx_text(filepath: str) -> str:
    doc = Document(filepath)
    text = []

    for paragraph in doc.paragraphs:
        if paragraph.text.strip():  # skip empty lines
            text.append(paragraph.text.strip())

    return "\n".join(text)

---
### Format ###

In [5]:
# ---- Reusable templates ----
format_template = """\
Epic: <summary>
  Description: <description>
  - Task: <summary>
    Description: <description>
    - Subtask: <summary>
      Description: <description>
    - Subtask: <summary>
      Description: <description>
  - Task: <summary>
    Description: <description>
    ...
"""

---
### Agent 1: Document Structure Agent ###
Role: Reads a text and creates tree-like structure in order to be used for the structure of Jira tickets.

In [6]:
def document_structure_agent(document_text: str, format_template: str, model: str = "gpt-4o-mini") -> str:

    prompt = f"""
    You are an expert in analyzing technical documents and creating structured plans.
    Given the following document, extract the high-level structure and identify the Epics, Tasks, and Subtasks.

    INSTRUCTIONS:
    1. For each item, provide:
       - A short **title/summary** (max 10 words, concise, like a Jira title).
       - A **Description** (1–3 sentences, expanded from the document's content).
    2. Use one Epic as parent for all the Tasks. The Epic Represents the hole document.
    3. For Tasks that concerns tabular reports, include all the fields needed in the description and 
       relate no Subtasks to this Tasks.
    4. Return the result in a hierarchical format with an Epic as the parent, followed by Tasks and then Subtasks.
    5. Only return the structure in the specified format with no explanation.

    Use the following format as guide:
    {format_template}

    Document:
    {document_text}
    """

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a precise, structured planning assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=3000
    )

    return resp.choices[0].message.content.strip()

---
### Agent 2: Review and Improve Structure ###
Role: Reviews the wotk of the previous agent and also processes the feedback from the user, if there is any.

In [7]:
def review_and_improve_structure(document_structure: str, format_template: str, feedback: str = "", model: str = "gpt-4o-mini") -> str:

    prompt = f"""
    You are an expert at reviewing and improving structured plans.

    The structure MUST stay in this hierarchy ONLY:
    - Epic
      - Task
        - Subtask

    !! Do not alter the format. Follow the one that is given.

    Review the work of a junior assistant and make sure it followed the instructions:
    1. For each item, provide:
       - A short **title/summary** (max 10 words, concise, like a Jira title).
       - A **Description** (1–3 sentences, expanded from the document's content).
    2. Use one Epic as parent for all the Tasks. The Epic Represents the hole document.
    3. For Tasks that concerns tabular reports, include all the fields needed in the description and 
       relate no Subtasks to this Tasks.
    4. Return the result in a hierarchical format with an Epic as the parent, followed by Tasks and then Subtasks.
    5. Only return the structure in the specified format with no explanation.
    6. IMPORTANT: Always prioritize and follow feedback provided. 
       If feedback conflicts with other instructions, 
       you must follow the feedback first, even if that means ignoring or overriding previous instructions.

    If feedback is provided, refine the structure accordingly. The feedback is:
    {feedback}

    Use the following format as guide:
    {format_template}

    Review and improve the following structure (output only the improved structure):
    {document_structure}
    """

    # Call GPT model via OpenAI API
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a meticulous Jira structure reviewer and improver."},
            {"role": "user", "content": prompt}
        ],
        temperature=0,
        max_tokens=3000
    )

    # Extract and return the improved structure
    revised_structure = response.choices[0].message.content.strip()
    return revised_structure if revised_structure else "No structured text returned. Please check the prompt or model."

---
### Staging Area: From Agent's Text to DataFrame ###

In [8]:
# ========================================================
# 1) Parse structure (Epic→Task→Subtask) with descriptions
# ========================================================

def parse_structure_text(structure_text: str) -> List[Dict[str, Any]]:

    epics: List[Dict[str, Any]] = []
    current_epic = None
    current_task = None
    current_subtask = None
    last_node = None

    raw_lines = structure_text.splitlines()
    lines = [ln.rstrip() for ln in raw_lines]

    def start_node(node_type: str, summary: str) -> Dict[str, Any]:
        node = {"summary": summary.strip(), "description": ""}
        if node_type == "epic":
            node["tasks"] = []
        elif node_type == "task":
            node["subtasks"] = []
        return node

    p_epic    = re.compile(r"^\s*(?:[-•]\s*)?Epic:\s*(.+)$", re.IGNORECASE)
    p_story   = re.compile(r"^\s*(?:[-•]\s*)?Story:\s*(.+)$", re.IGNORECASE)   # treated as Task
    p_task    = re.compile(r"^\s*(?:[-•]\s*)?Task:\s*(.+)$", re.IGNORECASE)
    p_subtask = re.compile(r"^\s*(?:[-•]\s*)?Sub[- ]?task:\s*(.+)$", re.IGNORECASE)
    p_desc    = re.compile(r"^\s*Description:\s*(.*)$", re.IGNORECASE)

    for ln in lines:
        if not ln.strip():
            if last_node is not None and last_node["description"]:
                last_node["description"] += "\n"
            continue

        line_no_bullet = re.sub(r"^\s*[-•]\s*", "", ln)

        m_epic = p_epic.match(ln)
        m_story = p_story.match(ln)
        m_task = p_task.match(ln)
        m_sub = p_subtask.match(ln)
        m_desc = p_desc.match(ln)

        if m_epic:
            current_epic = start_node("epic", m_epic.group(1))
            epics.append(current_epic)
            current_task = None
            current_subtask = None
            last_node = current_epic
            continue

        # Story lines are treated as Task lines
        if m_story or m_task:
            if not current_epic:
                current_epic = start_node("epic", "Miscellaneous Epic")
                epics.append(current_epic)
            title = (m_story or m_task).group(1)
            current_task = start_node("task", title)
            current_epic["tasks"].append(current_task)
            current_subtask = None
            last_node = current_task
            continue

        if m_sub:
            if not current_task:
                if not current_epic:
                    current_epic = start_node("epic", "Miscellaneous Epic")
                    epics.append(current_epic)
                current_task = start_node("task", "General Task")
                current_epic["tasks"].append(current_task)
            # leaf node for subtask
            current_subtask = {"summary": m_sub.group(1).strip(), "description": ""}
            current_task["subtasks"].append(current_subtask)
            last_node = current_subtask
            continue

        if m_desc:
            if last_node is not None:
                last_node["description"] = m_desc.group(1).strip()
            continue

        # Continuation of last description
        if last_node is not None:
            if last_node["description"]:
                last_node["description"] += ("\n" + line_no_bullet)
            else:
                last_node["description"] = line_no_bullet

    return epics

In [9]:
# ========================================
# 2) Flatten to rows for staging DataFrame
# ========================================

def flatten_to_staging_rows(parsed: List[Dict[str, Any]]) -> List[Dict[str, Any]]:

    rows: List[Dict[str, Any]] = []
    epic_i = task_i = sub_i = 0

    for epic in parsed:
        epic_i += 1
        e_id = f"E{epic_i}"
        rows.append({
            "local_id": e_id,
            "summary": epic.get("summary", ""),
            "description": epic.get("description", "").strip(),
            "issue_type": "Epic",
            "parent_local_id": ""
        })

        for task in epic.get("tasks", []):
            task_i += 1
            t_id = f"T{task_i}"
            rows.append({
                "local_id": t_id,
                "summary": task.get("summary", ""),
                "description": task.get("description", "").strip(),
                "issue_type": "Task",
                "parent_local_id": e_id
            })

            for sub in task.get("subtasks", []):
                sub_i += 1
                u_id = f"U{sub_i}"
                rows.append({
                    "local_id": u_id,
                    "summary": sub.get("summary", ""),
                    "description": sub.get("description", "").strip(),
                    "issue_type": "Subtask",   # exact spelling per your project
                    "parent_local_id": t_id
                })

    return rows

In [10]:
# ======================
# 3) Built the DataFrame
# ======================

def build_staging_dataframe(structure_text: str) -> pd.DataFrame:
    parsed = parse_structure_text(structure_text)  # your Description-aware parser
    rows = flatten_to_staging_rows(parsed)
    # Stable, Jira-ready columns
    return pd.DataFrame(rows, columns=[
        "local_id", "summary", "description", "issue_type", "parent_local_id"
    ])

---
### User Feedback and Approve ###
Role: Prints Outcome and the user gives feedback. If it is not okey, it sends the feedback back to the first agent to start agent with the extra instructions.

In [11]:
def get_user_feedback_and_approve(structure: str) -> Tuple[bool, Optional[pd.DataFrame]]:

    while True:
        # Always build fresh from the current structure
        df = build_staging_dataframe(structure)

        print("Here is the Jira ticketing structure:\n")
        display(df)
        #show_jira_hierarchy_tree(df, direction="UD")

        feedback = input("Are you okay with this structure? (yes/no): ").strip().lower()

        if feedback == "yes":
            print("\nThe process continues.")
            return True, df

        elif feedback == "no":
            print("Please provide feedback on the changes you would like to make.")
            user_feedback = input("Your feedback: ")

            # Refine the structure with the feedback
            structure = review_and_improve_structure(structure, format_template, feedback=user_feedback)

            print("\nThe structure was refined after feedback.\n")
            #print(structure)

            # loop continues — will rebuild df in next iteration

        else:
            print("⚠️ Please answer 'yes' or 'no'.")

---
### Agent 3: Add Labels ###
Role: Adds labels to Jira tickets to help progress/work tracking and also help the project manager and teams understand the responsibilities for each section of the project.

In [12]:
labels_list = [
    "QA", "Reporting", "ETL", "Data", 
    "Development", "Documentation", "Business"
]

In [13]:
def add_labels_to_dataframe(df: pd.DataFrame, model: str = "gpt-4o-mini") -> pd.DataFrame:

    df = df.copy()
    df["label"] = None  # new column

    for idx, row in df.iterrows():
        context = f"Summary: {row['summary']}\nDescription: {row['description']}"

        prompt = f"""
        You are an expert at categorizing work items.

        Choose ONE label from this list based on the content:
        {", ".join(labels_list)}

        Rules:
        - Return only the label (exact spelling, no extra text).
        - If multiple could apply, choose the most relevant one.
        - Context is from Jira-style summary and description.

        Context:
        {context}
        """

        resp = client.chat.completions.create(
            model=model,
            temperature=0,
            max_tokens=1000,
            messages=[
                {"role": "system", "content": "You are a precise classifier that returns only one label."},
                {"role": "user", "content": prompt}
            ]
        )

        label = resp.choices[0].message.content.strip()
        df.at[idx, "label"] = label

    return df

---
### Agent 4: Assigns Ticket to Group ###
Role: Assigns Jira ticket to a Group, based on the descirption and the label.

In [14]:
groups_list = [
    "Business Analysis Team",
    "Business Intelligence Team",
    "Configuration Engineer Team",
    "Data Warehouse Team",
    "Project Management Office (PMO)",
    "Quality Assurance Team"
]

In [15]:
def add_groups_to_dataframe(df: pd.DataFrame, model: str = "gpt-4o-mini") -> pd.DataFrame:

    df_copy = df.copy()
    df_copy["assignee_group"] = None

    for idx, row in df_copy.iterrows():
        summary = (row.get("summary") or "").strip()
        description = (row.get("description") or "").strip()
        label = (row.get("label") or "").strip() if "label" in df_copy.columns else ""

        context = f"Summary: {summary}\nDescription: {description}"
        if label:
            context += f"\nLabel: {label}"

        prompt = f"""
        You are assigning each work item to exactly ONE delivery group.
        
        Choose ONE group from this list based on the content:
        {", ".join(groups_list)}
        
        Guidelines:
        - Use the summary, description, and label (if provided).
        - Choose the best single fit.
        - Return ONLY the group name, with exact casing and spelling. No extra words.
        
        Item:
        {context}
        """

        resp = client.chat.completions.create(
            model=model,
            temperature=0,
            max_tokens=20,
            messages=[
                {"role": "system", "content": "You are a precise triage assistant. Output exactly one allowed group."},
                {"role": "user", "content": prompt}
            ],
        )

        label = resp.choices[0].message.content.strip()
        df.at[idx, "label"] = label
        
        group = resp.choices[0].message.content.strip()
        df_copy.at[idx, "assignee_group"] = group

    return df_copy

---
### Agent 5: Find Missing Columns ###
Role: Reads the document and the database and returns a list of what it thinks are the missing columns.

In [16]:
# Load the Database Schema
def load_schema_txt(path: str | Path) -> str:
    return Path(path).read_text(encoding="utf-8")

In [17]:
def find_missing_fields_df_from_staging(df: pd.DataFrame, schema_text: str, model: str = "gpt-4o-mini") -> pd.DataFrame:

    # --- validate required columns
    required_cols = {"local_id", "summary", "description", "label"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Input df must contain columns: {missing}")

    # --- filter only Reporting
    df_filtered = df[df["label"].astype(str).str.strip().str.lower() == "reporting"]

    rows_out = []

    for _, row in df_filtered.iterrows():
        local_id    = str(row.get("local_id", "")).strip()
        summary     = (row.get("summary") or "").strip()
        description = (row.get("description") or "").strip()

        prompt = f"""
        You analyze report specifications against an existing database schema and identify missing fields.
        
        SCHEMA (human-readable text):
        {schema_text.strip()}
        
        ITEM CONTEXT:
        Summary: {summary}
        Description: {description}
        
        TASK:
        - If this report clearly requires columns NOT present in the schema, list them.
        - For each missing field, return:
          • missing_field_name (as mentioned in the report)
          • suggested_table (existing table if appropriate, else a sensible new table name)
        - If nothing is missing, return an empty array.
        
        OUTPUT:
        Return ONLY valid JSON array, like:
        [
          {{"missing_field_name":"CustomerSegment","suggested_table":"customers"}}
        ]
        """.strip()

        resp = client.chat.completions.create(
            model=model,
            temperature=0,
            max_tokens=800,
            messages=[
                {"role": "system", "content": "You compare report requirements to schemas and output only valid JSON arrays."},
                {"role": "user", "content": prompt}
            ],
        )

        text = (resp.choices[0].message.content or "").strip()

        # parse JSON array; fallback: extract first array
        try:
            data = json.loads(text)
        except Exception:
            m = re.search(r"\[\s*{.*}\s*\]", text, flags=re.DOTALL)
            data = json.loads(m.group(0)) if m else []

        if isinstance(data, list):
            for item in data:
                rows_out.append({
                    "missing_field_name":   (item.get("missing_field_name") or "").strip(),
                    "suggested_table": (item.get("suggested_table") or "").strip(),
                    "parent_local_id":      local_id
                })

    # --- build initial DF
    out = pd.DataFrame(rows_out, columns=[
        "missing_field_name", "suggested_table", "parent_local_id"
    ])

    # If empty, return as-is
    if out.empty:
        return out

    # --- normalize missing_field_name to CamelCase (no spaces/underscores); no special Number→No rule
    def camelize(name: str) -> str:
        tokens = re.findall(r"[A-Za-z0-9]+", (name or ""))
        # Preserve all-caps acronyms (e.g., ID), otherwise Capitalize
        parts = [t if t.isupper() and len(t) <= 4 else t.capitalize() for t in tokens]
        return "".join(parts)

    out["missing_field_name"] = out["missing_field_name"].apply(camelize)

    # --- deduplicate by normalized missing_field_name, keeping the first occurrence (first report wins)
    out = out.drop_duplicates(subset=["missing_field_name"], keep="first").reset_index(drop=True)

    return out

---
### Function: Add Subtasks to main DataFrame ###

In [18]:
def append_add_field_subtasks_interactive(
    fields_df: pd.DataFrame,
    staging_df: pd.DataFrame,
    *,
    default_label: str = "Data",
    default_group: str = "Data Warehouse Team"
) -> pd.DataFrame:

    required_cols = {"missing_field_name", "suggested_table", "parent_local_id"}
    missing = required_cols - set(fields_df.columns)
    if missing:
        raise ValueError(f"fields_df missing columns: {missing}")

    df = staging_df.copy()

    # Helper to get next available Subtask local_id
    def next_sub_id(existing_ids: pd.Series) -> str:
        max_n = 0
        for v in existing_ids.dropna().astype(str):
            m = re.match(r"U(\d+)$", v.strip(), re.IGNORECASE)
            if m:
                max_n = max(max_n, int(m.group(1)))
        return f"U{max_n + 1}"

    for _, row in fields_df.iterrows():
        field_name = str(row["missing_field_name"]).strip()
        table_name = str(row["suggested_table"]).strip()
        parent_id = str(row["parent_local_id"]).strip()

        # Step 1: Check if field exists in DB
        exists_ans = input(f"Does field '{field_name}' already exist in the database? (yes/no): ").strip().lower()
        if exists_ans == "yes":
            print(f"Skipping '{field_name}' — already exists.")
            continue

        # Step 2: Ask if we should add it
        add_ans = input(f"Do you want to add '{field_name}' to table '{table_name}' as a subtask? (yes/no): ").strip().lower()
        if add_ans != "yes":
            print(f"Skipping '{field_name}' — not adding as subtask.")
            continue

        # Step 3: Create the new subtask
        new_id = next_sub_id(df["local_id"]) if "local_id" in df.columns else "U1"
        summary = f'Add field "{field_name}" to table "{table_name}"'
        description = f'Implement changes to add the field "{field_name}" into table "{table_name}", including ETL and database updates.'

        new_row = {
            "local_id": new_id,
            "summary": summary,
            "description": description,
            "issue_type": "Subtask",
            "parent_local_id": parent_id,
            "label": default_label,
            "assignee_group": default_group
        }

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    return df

---
### Attach Subtask local_id ###

In [19]:
def attach_subtask_local_ids(fields_df: pd.DataFrame, staging_df: pd.DataFrame) -> pd.DataFrame:

    need_f = {"missing_field_name", "suggested_table", "parent_local_id"}
    missing = need_f - set(fields_df.columns)
    if missing:
        raise ValueError(f"fields_df missing columns: {missing}")

    need_s = {"local_id", "summary", "issue_type", "parent_local_id"}
    missing_s = need_s - set(staging_df.columns)
    if missing_s:
        raise ValueError(f"staging_df missing columns: {missing_s}")

    out = fields_df.copy()
    out["local_id"] = ""

    # Filter only subtasks once for speed
    subs = staging_df[staging_df["issue_type"] == "Subtask"][["local_id","summary","parent_local_id"]].copy()

    for idx, r in out.iterrows():
        fname = str(r["missing_field_name"]).strip()
        tname = str(r["suggested_table"]).strip()
        parent = str(r["parent_local_id"]).strip()
        expected_summary = f'Add field "{fname}" to table "{tname}"'

        hit = subs[(subs["summary"] == expected_summary) & (subs["parent_local_id"].astype(str) == parent)]
        if not hit.empty:
            out.at[idx, "local_id"] = str(hit.iloc[0]["local_id"])
        else:
            # Not found — leave local_id empty so you can spot & fix
            pass

    return out

---
### Agent 6: Generate SQL Script for Changing the Database Scheme ###
Role: Generates SQL script regarding the alteration of database scheme in order to attach it in the Jira ticket.

In [20]:
def generate_alter_sql_per_field_from_staging(
    fields_df: pd.DataFrame,
    staging_df: pd.DataFrame,
    schema_text: str,
    model: str = "gpt-4o-mini"
) -> pd.DataFrame:

    # First, enrich with subtask local_id
    merged = attach_subtask_local_ids(fields_df, staging_df)

    out_rows = []
    for _, r in merged.iterrows():
        sub_local_id = (r.get("local_id") or "").strip()
        if not sub_local_id:
            # Skip if we can't find the subtask row
            continue

        field_name = str(r["missing_field_name"]).strip()
        table_name = str(r["suggested_table"]).strip()

        prompt = f"""
        You write concise ANSI SQL DDL.
        
        Schema (human-readable for context):
        {schema_text}
        
        Task:
        Write ONE single-line ALTER TABLE statement to add the column below to the target table.
        - If type/length is unclear, choose a sensible default (e.g., VARCHAR(100) or BOOLEAN or DATE).
        - Default to NULL unless clearly NOT NULL.
        - Do NOT include comments or code fences. End with a semicolon.
        
        Column to add:
        - Table: {table_name}
        - Column: {field_name}
        
        Output:
        A single ALTER TABLE ... ADD ...; statement, nothing else.
        """.strip()

        resp = client.chat.completions.create(
            model=model,
            temperature=0,
            max_tokens=200,
            messages=[
                {"role": "system", "content": "Return only one ANSI SQL ALTER statement; no explanations."},
                {"role": "user", "content": prompt}
            ],
        )

        sql = (resp.choices[0].message.content or "").strip()
        # Clean any accidental code fences/spaces
        sql = re.sub(r"^```[\s\S]*?\n", "", sql)
        sql = re.sub(r"\n```$", "", sql)
        sql = sql.strip()
        if not sql.endswith(";"):
            sql += ";"

        out_rows.append({
            "local_id": sub_local_id,
            "field_name": field_name,
            "suggested_table": table_name,
            "sql_script": sql
        })

    return pd.DataFrame(out_rows, columns=["local_id", "field_name", "suggested_table", "sql_script"])

---
### Agent 7: Identify Tabular Reports ###
Role: Reads the summary and the description for the reporting Tasks and decides which one concerns Tabular reports in order to pass it to the next Agent and create the SELECT Statement.

In [21]:
def identify_tabular_reports(staging_df: pd.DataFrame, model: str = "gpt-4o-mini") -> pd.DataFrame:

    rows = []
    for _, r in staging_df.iterrows():
        #if str(r.get("issue_type", "")).strip().lower() != "task":
            #rows.append(False)
            #continue
        if str(r.get("label", "")).strip().lower() != "reporting":
            rows.append(False)
            continue

        context = f"Summary: {r.get('summary','')}\nDescription: {r.get('description','')}"
        
        prompt = f"""
        You are an analyst.
        Determine if the following describes a TABULAR REPORT (e.g. SELECT query with multiple fields).
        Respond ONLY with true or false.

        CONTEXT:
        {context}
        """

        resp = client.chat.completions.create(
            model=model,
            temperature=0,
            max_tokens=5,
            messages=[
                {"role": "system", "content": "Return only 'true' or 'false'."},
                {"role": "user", "content": prompt}
            ],
        )

        text = (resp.choices[0].message.content or "").strip().lower()
        rows.append(text.startswith("t"))

    staging_df["is_tabular_report"] = rows
    return staging_df

---
### Agent 8: Generate SELECT Statement for Tabular Reports ###
Role: Takes the Tasks from the previous Agent, and based on the description, it generates a SELECT Statement for the Tabular report using the Database Scheme and the missing fields from a previous Agent.

In [22]:
def generate_select_sql_for_reports(
    staging_df: pd.DataFrame,
    schema_text: str,
    fields_df: pd.DataFrame | None = None,   # optional
    model: str = "gpt-4o-mini"
) -> pd.DataFrame:

    out_rows = []

    # Build quick lookup: parent_local_id -> [(field, table), ...]
    field_map: dict[str, list[tuple[str, str]]] = {}
    if fields_df is not None and not fields_df.empty:
        for _, f in fields_df.iterrows():
            parent = str(f.get("parent_local_id") or "").strip()
            field_map.setdefault(parent, []).append(
                (str(f.get("missing_field_name") or "").strip(),
                 str(f.get("suggested_table") or "").strip())
            )

    for _, r in staging_df.iterrows():
        if not bool(r.get("is_tabular_report")):
            continue

        local_id = str(r["local_id"])
        context = f"Summary: {r.get('summary','')}\nDescription: {r.get('description','')}"

        # Attach schema mapping info if available for this task/local_id
        extra_fields = field_map.get(local_id, [])
        mapping_text = ""
        if extra_fields:
            lines = [f"- {col} (from {tbl})" for col, tbl in extra_fields if col and tbl]
            if lines:
                mapping_text = "\nMapped fields from schema:\n" + "\n".join(lines)

        prompt = f"""
        You are a SQL expert.
        Write a realistic ANSI SQL SELECT statement for the following tabular report.
        Use tables and joins consistent with this schema:
        {schema_text}
        {mapping_text}

        Report requirement:
        {context}     

        Output:
        - Return ONLY the SQL query (no explanation, no markdown).
        - Include sensible JOINs based on the schema.
        - Try using fields from the {schema_text} or {mapping_text} before making calculation using fields from the {schema_text}.
        - Use aliases when helpful.
        """.strip()

        resp = client.chat.completions.create(
            model=model,
            temperature=0,
            max_tokens=600,
            messages=[
                {"role": "system", "content": "Return only the SQL query; no extra text."},
                {"role": "user", "content": prompt}
            ],
        )

        sql = (resp.choices[0].message.content or "").strip()
        # Remove code fences if any
        sql = re.sub(r"^```sql\s*", "", sql, flags=re.IGNORECASE)
        sql = re.sub(r"^```", "", sql)
        sql = re.sub(r"```$", "", sql)
        sql = sql.strip()

        # Convert literal \n sequences to actual newlines for readability & better attachments
        sql = sql.replace("\\n", "\n").strip()

        # Optionally normalize multiple blank lines
        sql = re.sub(r"\n{3,}", "\n\n", sql)

        out_rows.append({"local_id": local_id, "sql_script": sql})

    if out_rows:
        sql_df = pd.DataFrame(out_rows)

        # Ensure sql_script column exists
        if "sql_script" not in staging_df.columns:
            staging_df["sql_script"] = None

        # Merge and only fill missing values (don’t overwrite existing scripts)
        staging_df = staging_df.merge(sql_df, on="local_id", how="left", suffixes=("", "_new"))
        staging_df["sql_script"] = staging_df["sql_script"].combine_first(staging_df["sql_script_new"])
        staging_df = staging_df.drop(columns=["sql_script_new"], errors="ignore")

    return staging_df

---
### Upload Attachments to Jira ###

In [23]:
def upload_attachment_to_issue(issue_key: str, file_path: str) -> bool:
    url = f"{JIRA_BASE_URL}/rest/api/3/issue/{issue_key}/attachments"
    headers = {"Accept": "application/json", "X-Atlassian-Token": "no-check"}
    auth = HTTPBasicAuth(JIRA_EMAIL, JIRA_API_TOKEN)
    with open(file_path, "rb") as f:
        files = {"file": (os.path.basename(file_path), f, "application/octet-stream")}
        r = requests.post(url, headers=headers, auth=auth, files=files)
    ok = r.status_code in (200, 201)
    if ok:
        print(f"\nAttached {os.path.basename(file_path)} → {issue_key}")
    else:
        try:
            print("Attachment failed:", r.status_code, r.json())
        except Exception:
            print("Attachment failed:", r.status_code, r.text)
    return ok

---
### Agent 9: Infer Priority per Row ###
Role: Creates/updates a priority column using statuses dynamically.

In [24]:
allowed = ["Highest",
           "High",
           "Medium",
           "Low",
           "Lowest"
          ]

In [25]:
def add_priority_predictions(df: pd.DataFrame, model: str = "gpt-4o-mini") -> pd.DataFrame:

    df2 = df.copy()

    by_local = {str(r.local_id): r for _, r in df2.iterrows()}
    child_count = {}
    for _, r in df2.iterrows():
        parent = str(r.get("parent_local_id") or "").strip()
        if parent:
            child_count[parent] = child_count.get(parent, 0) + 1

    priorities = []
    for _, r in df2.iterrows():
        local_id     = str(r.get("local_id") or "")
        summary      = (r.get("summary") or "").strip()
        description  = (r.get("description") or "").strip()
        issue_type   = (r.get("issue_type") or "").strip()
        label        = (r.get("label") or "").strip()
        parent_id    = (r.get("parent_local_id") or "").strip()

        parent_type  = (by_local.get(parent_id).issue_type if parent_id in by_local else "") or ""
        parent_sum   = (by_local.get(parent_id).summary if parent_id in by_local else "") or ""
        has_children = child_count.get(local_id, 0) > 0

        context = f"""IssueType: {issue_type}
        Summary: {summary}
        Description: {description}
        Label: {label}
        Parent: {parent_id} ({parent_type}) {parent_sum}
        HasChildren: {has_children}"""

        prompt = f"""
        You are a Jira triage assistant. Assign a PRIORITY to this work item.
        
        Allowed values (choose ONE only):
        Highest | High | Medium | Low | Lowest
        
        Guidelines:
        - Highest: blocker, report fails completely, business cannot continue, schema changes needed urgently.
        - High: severe issue, report inaccurate or critical ETL, must be resolved quickly.
        - Medium: normal functional requirement, important but not blocking other work.
        - Low: cosmetic, minor enhancement, documentation, or low impact.
        - Lowest: trivial, "nice to have", optional extras.
        
        Important:
        - Do NOT default everything to Medium.
        - Think carefully about urgency, dependencies, and business impact.
        - Output must be exactly one of: Highest, High, Medium, Low, Lowest.
        
        Item:
        {context}
        """

        resp = client.chat.completions.create(
            model=model,
            temperature=0,
            max_tokens=5,
            messages=[
                {"role":"system","content":"Return only one of: Highest | High | Medium | Low | Lowest"},
                {"role":"user","content":prompt}
            ],
        )
        raw = (resp.choices[0].message.content or "").strip()
        choice = next((p for p in allowed if p.lower() == raw.lower()), None)
        priorities.append(choice)

    df2["priority"] = priorities
    return df2

---
### Function: Ask Final Approval

In [26]:
def ask_final_approval(df: pd.DataFrame) -> Tuple[bool, Optional[str]]:

    print("\nHere is the final ticketing structure:")
    # Hide super-long sql text by default when printing
    if "sql_script" in df.columns:
        display(df.drop(columns=["sql_script","is_tabular_report"]))
    else:
        display(df)

    while True:
        ans = input("Are you okay with this final outcome? (yes/no): ").strip().lower()
        if ans in ("yes", "no"):
            break
        print("⚠️ Please answer 'yes' or 'no'.")

    if ans == "yes":
        return True, None

    print("Please provide feedback on the changes you would like to make (values only, not structure).")
    fb = input("Your feedback: ").strip()
    return False, fb or None

---
### Agent 10: Apply Feedback to DataFrame
Role: Applies feedback to dataframe but it is not changing the structure.

In [27]:
def apply_feedback_to_dataframe_values(
    df: pd.DataFrame,
    feedback: str,
    *,
    model: str = "gpt-4o-mini",
    allowed_columns: Optional[List[str]] = None,
    extra_context_columns: Optional[List[str]] = None,
    temperature: float = 0.0
) -> pd.DataFrame:

    if allowed_columns is None:
        # Safe defaults: edit categorization/text but not identifiers or structure
        allowed_columns = ["label", "assignee_group", "priority", "summary", "description"]

    if extra_context_columns is None:
        extra_context_columns = ["label", "assignee_group", "priority"]

    protected = {"local_id", "issue_type", "parent_local_id"}

    df2 = df.copy()

    # Ensure we never try to edit protected columns
    allowed_columns = [c for c in allowed_columns if c not in protected and c in df2.columns]

    system_msg = (
        "You are a careful data editor. You will update ONLY the specified columns in a row, "
        "following the user's feedback. Do not add or remove rows. "
        "Never change 'local_id', 'issue_type', or 'parent_local_id'. "
        "Return a JSON object with only the keys that need to be updated for this row. "
        "If no changes are required, return an empty JSON object {}."
    )

    instructions = f"""
    User feedback (apply globally where appropriate, but decide per-row by context):
    {feedback}
    
    Rules:
    - Do NOT modify: local_id, issue_type, parent_local_id.
    - You MAY modify only these columns if present: {allowed_columns}.
    - Use the row context (summary + description and any extra fields) to decide.
    - Output MUST be a SINGLE JSON object with only the fields to change for this row, e.g.:
      {{"label":"Data","assignee_group":"Data Warehouse Team"}}
    - If no change needed, output: {{}}
    """

    # Process row-by-row
    updates: List[Dict[str, Any]] = []
    for _, row in df2.iterrows():
        row_dict = row.to_dict()

        # Build compact context for the model
        ctx = {
            "local_id": row_dict.get("local_id"),
            "issue_type": row_dict.get("issue_type"),
            "parent_local_id": row_dict.get("parent_local_id"),
            "summary": row_dict.get("summary", ""),
            "description": row_dict.get("description", ""),
        }
        for c in extra_context_columns:
            if c in row_dict:
                ctx[c] = row_dict[c]

        # Send to model
        user_msg = (
            "Row context (use this to understand the row; do not change these directly):\n"
            + json.dumps(ctx, ensure_ascii=False)
            + "\n\n"
            "Allowed output keys (only if changing): "
            + ", ".join(allowed_columns)
            + "\n\n"
            "Return only a single JSON object (no code fences, no extra text)."
        )

        resp = client.chat.completions.create(
            model=model,
            temperature=temperature,
            max_tokens=150,
            messages=[
                {"role": "system", "content": system_msg},
                {"role": "user", "content": instructions},
                {"role": "user", "content": user_msg},
            ],
        )

        raw = (resp.choices[0].message.content or "").strip()

        # Be robust to accidental code fences
        raw = re.sub(r"^```json\s*", "", raw, flags=re.IGNORECASE).strip()
        raw = re.sub(r"^```", "", raw).strip()
        raw = re.sub(r"```$", "", raw).strip()

        try:
            delta = json.loads(raw)
            if not isinstance(delta, dict):
                delta = {}
        except Exception:
            delta = {}

        # Apply only allowed columns
        clean_delta = {k: v for k, v in delta.items() if k in allowed_columns}
        updates.append(clean_delta)

    # Merge updates back
    for i, upd in enumerate(updates):
        if upd:
            for k, v in upd.items():
                df2.iat[i, df2.columns.get_loc(k)] = v

    return df2

---
### Create Jira tickets ###

In [28]:
def create_jira_issue(summary: str,
                      description: str | None,
                      issue_type: str,                 # "Epic" | "Task" | "Subtask" | etc.
                      parent_key: str | None = None,   # Epic for Tasks (team-managed) or Task for Subtasks
                      labels: list[str] | None = None, # Jira standard labels (array of strings)
                      assigned_group: str | None = None,  # Your “Assigned Group” field (single value)
                      custom_fields: dict | None = None,
                      priority_name: str | None = None   # <-- added
                      ) -> str | None:

    # ---- helpers ----
    def _auth(): return HTTPBasicAuth(JIRA_EMAIL, JIRA_API_TOKEN)
    def _headers(): return {"Accept":"application/json","Content-Type":"application/json"}
    def _to_adf(text):
        if not text: return None
        blocks=[]
        for line in str(text).split("\n"):
            if line.strip():
                blocks.append({"type":"paragraph","content":[{"type":"text","text":line}]})
            else:
                blocks.append({"type":"paragraph","content":[]})
        while blocks and blocks[-1].get("content")==[]:
            blocks.pop()
        if not blocks: blocks=[{"type":"paragraph"}]
        return {"type":"doc","version":1,"content":blocks}

    def _post(fields):
        url=f"{JIRA_BASE_URL}/rest/api/3/issue"
        return requests.post(url, json={"fields":fields}, auth=_auth(), headers=_headers())

    # Normalize a Jira label (no spaces)
    def _norm_label(s: str) -> str:
        s = (s or "").strip().lower()
        s = re.sub(r"\s+", "-", s)
        s = re.sub(r"[^a-z0-9._-]", "", s)
        return s

    # ---- normalize issuetype ----
    t=issue_type.strip().lower()
    if t in ("sub-task","subtask"): issuetype_name="Subtask"
    elif t=="story":                 issuetype_name="Task"  # if you’ve removed stories
    else:                            issuetype_name=issue_type

    # ---- base fields ----
    fields={
        "project":{"key":JIRA_PROJECT_KEY},
        "summary":summary,
        "issuetype":{"name":issuetype_name},
    }
    adf=_to_adf(description)
    if adf: fields["description"]=adf

    # Priority (standard Jira field)
    if priority_name and priority_name.strip():
        fields["priority"] = {"name": priority_name.strip()}

    # Jira labels (standard field expects list[str])
    if labels:
        safe_labels = [_norm_label(x) for x in labels if isinstance(x, str) and x.strip()]
        if safe_labels:
            fields["labels"] = safe_labels

    # Optional Epic Name (rarely needed on Cloud Epics)
    if issuetype_name.lower()=="epic" and USE_EPIC_NAME and JIRA_EPIC_NAME_FIELD:
        fields[JIRA_EPIC_NAME_FIELD]=summary

    # Custom field example (Suggested Script) on non-Epic
    if custom_fields and issuetype_name.lower()!="epic":
        if JIRA_FIELD_SUGGESTED_SCRIPT and "Suggested Script" in custom_fields:
            val=custom_fields["Suggested Script"]
            if isinstance(val,str) and val.strip():
                fields[JIRA_FIELD_SUGGESTED_SCRIPT]=val.strip()

    # Assigned Group (custom field). We’ll try common payload shapes:
    # 1) Select list: {"value": "<Group Name>"}
    # 2) Group picker: {"name": "<Group Name>"}  (fallback on retry if needed)
    JIRA_FIELD_ASSIGNED_GROUP = os.getenv("JIRA_FIELD_ASSIGNED_GROUP", "")
    assigned_group_value = (assigned_group or "").strip()
    if JIRA_FIELD_ASSIGNED_GROUP and assigned_group_value:
        fields[JIRA_FIELD_ASSIGNED_GROUP] = {"value": assigned_group_value}

    # ---- team-managed style: set parent for ANY non-epic if provided ----
    # (Tasks → Epic, Subtasks → Task)
    if parent_key and issuetype_name.lower()!="epic":
        fields["parent"]={"key":parent_key}

    # ---- create (attempt 1) ----
    r=_post(fields)
    if r.status_code==201:
        key=r.json().get("key")
        print(f"Created {issuetype_name}: {key}")
        return key

    # If server rejects 'parent' for Tasks (company-managed), retry without parent
    retry_parent=False
    retry_assigned_group=False
    try:
        body=r.json()
        if isinstance(body,dict) and "errors" in body:
            # parent not allowed on standard issues
            if "parent" in body["errors"] or any("parent" in (str(v) or "") for v in body["errors"].values()):
                retry_parent=True
            # custom assigned group field rejected -> try alternate shape
            if JIRA_FIELD_ASSIGNED_GROUP and JIRA_FIELD_ASSIGNED_GROUP in body["errors"]:
                retry_assigned_group=True
    except Exception:
        pass

    # Prepare second attempt payload if needed
    fields_retry = dict(fields)

    # If assigned group failed with {"value": ...}, try {"name": ...}
    if retry_assigned_group and JIRA_FIELD_ASSIGNED_GROUP and assigned_group_value:
        ag = fields_retry.get(JIRA_FIELD_ASSIGNED_GROUP)
        if isinstance(ag, dict) and "value" in ag:
            fields_retry[JIRA_FIELD_ASSIGNED_GROUP] = {"name": assigned_group_value}

    # If parent not allowed for non-subtask, drop it and retry
    if retry_parent and parent_key and issuetype_name.lower()!="subtask":
        fields_retry.pop("parent", None)

    if retry_parent or retry_assigned_group:
        r2=_post(fields_retry)
        if r2.status_code==201:
            key=r2.json().get("key")
            #print(f"Created {issuetype_name} (retry adjustments): {key}")
            return key
        else:
            try: body2=r2.json()
            except Exception: body2=r2.text
            print(f"Failed to create {issuetype_name} on retry. Status:", r2.status_code)
            print("Response:", body2)
            return None

    # final failure
    try: body=r.json()
    except Exception: body=r.text
    print(f"Failed to create {issuetype_name}. Status:", r.status_code)
    print("Response:", body)
    return None

In [36]:
def create_jira_from_df(df: pd.DataFrame, *, attach_sql: bool = True, sql_dir: str = "sql_scripts") -> dict:

    required = {"local_id", "summary", "description", "issue_type", "parent_local_id"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Staging DataFrame missing columns: {missing}")

    with pd.option_context("display.max_colwidth", 200, "display.width", 160):
        print("\nHere is the Jira ticketing structure, with the extra subtasks:\n")
        display(df.drop(columns=["sql_script", "is_tabular_report"], errors="ignore"))

    id_map: dict[str, str] = {}

    # -- small local helpers --
    def _row_label(row):
        if "label" in df.columns:
            val = (row.get("label") or "").strip()
            return [val] if val else None
        return None

    def _row_group(row):
        if "assignee_group" in df.columns:
            val = row.get("assignee_group")
            if isinstance(val, str):
                return val.strip() or None
            return None
        return None

    def _slug(s: str) -> str:
        s = (s or "").strip().lower()
        s = re.sub(r"[^a-z0-9]+", "_", s)
        return re.sub(r"_+", "_", s).strip("_") or "script"

    def _save_sql_file(jira_key: str, summary: str, sql_text: str) -> str:
        """Save SQL content to <sql_dir>/<jira_key>_<slug(summary)>.sql and return path."""
        Path(sql_dir).mkdir(parents=True, exist_ok=True)
        fname = f"{jira_key}_{_slug(summary)}.sql"
        path = Path(sql_dir) / fname
        sql_clean = sql_text.rstrip()
        if not sql_clean.endswith(";"):
            sql_clean += ";"
        path.write_text(sql_clean + "\n", encoding="utf-8")
        return str(path)

    def _priority(row):
        return (row.get("priority") or "").strip() or None

    def _create_issue_with_sql(row, issue_type: str, *, parent_key: str | None = None, include_assignee_group: bool = True):
        """Helper: create issue + attach sql if exists"""
        assignee_group = _row_group(row) if include_assignee_group else None
        jira_key = create_jira_issue(
            summary=(row.summary or "").strip(),
            description=(row.description or "").strip(),
            issue_type=issue_type,
            parent_key=parent_key,
            labels=_row_label(row),
            assigned_group=assignee_group,
            priority_name=_priority(row)
        )
        id_map[str(row.local_id)] = jira_key
        #print(f"\n{issue_type} created:", jira_key)

        # Attach SQL if requested and available
        if attach_sql and jira_key and "sql_script" in df.columns:
            raw_sql_val = row.get("sql_script", None)
            if not pd.isna(raw_sql_val):
                sql_text = str(raw_sql_val).strip()
                if sql_text and sql_text.lower() != "nan":
                    try:
                        file_path = _save_sql_file(jira_key, (row.summary or ""), sql_text)
                        upload_attachment_to_issue(jira_key, file_path)
                    except Exception as ex:
                        print(f"Warning: failed to attach SQL for {jira_key}: {ex}")

        return jira_key

    # 1) Epics
    for _, e in df[df.issue_type == "Epic"].iterrows():
        _create_issue_with_sql(e, "Epic")

    # 2) Tasks (parent = Epic)
    for _, t in df[df.issue_type == "Task"].iterrows():
        parent_local = str(t.parent_local_id or "")
        epic_key = id_map.get(parent_local)
        _create_issue_with_sql(t, "Task", parent_key=epic_key)

    # 3) Subtasks (parent = Task)
    for _, s in df[df.issue_type == "Subtask"].iterrows():
        parent_local = str(s.parent_local_id or "")
        parent_task_key = id_map.get(parent_local)
        _create_issue_with_sql(s, "Subtask", parent_key=parent_task_key)

    return id_map

---
---

### Final Pipeline ###

In [37]:
def run_pipeline(
    doc_path: str,
    schema_text: str | None,
    include_sql: bool = True,
    include_assignee_group: bool = True
):
    # ===== 1) STRUCTURE APPROVAL LOOP =====
    document_text = read_docx_text(doc_path)
    staging_df = None
    approved = False

    while not approved:
        # Agent 1: Parse
        structure = document_structure_agent(document_text, format_template)

        # Agent 2: Review
        improved_structure = review_and_improve_structure(structure, format_template)

        # Ask user to approve the improved structure (returns (approved_bool, df OR None))
        approved, staging_df = get_user_feedback_and_approve(improved_structure)

    # ===== 2) ENRICHMENTS (labels, optional groups, optional SQL, priorities) =====
    print("\n🏷️ Adding labels with LLM...")
    staging_df = add_labels_to_dataframe(staging_df)  # adds 'label'

    if include_assignee_group:
        print("\n👥 Adding assignee group with LLM...")
        staging_df = add_groups_to_dataframe(staging_df)  # adds 'assignee_group'
    else:
        print("\n[INFO] Skipping assignee group enrichment.")

    # ---------- Optional SQL related steps ----------
    if include_sql:
        if not schema_text:
            print("\n[WARN] include_sql=True but no schema_text provided. Skipping SQL steps.")
        else:
            # Find missing fields in reporting tasks
            print("\n🔍 Scanning for missing report fields against schema...")
            fields_df = find_missing_fields_df_from_staging(staging_df, schema_text)
            if not fields_df.empty:
                with pd.option_context("display.max_colwidth", None, "display.width", 160):
                    print("\nSuggested missing fields:")
                    display(fields_df)

                print("\nConfirm which ones to append as standardized 'Add field to table' subtasks...")
                if include_assignee_group:
                    staging_df = append_add_field_subtasks_interactive(fields_df, staging_df)
                else:
                    staging_df = append_add_field_subtasks_interactive(fields_df, staging_df, default_group = "")
            else:
                print("No missing fields detected by the agent.")

            # Parse standardized subtasks into (missing_field_name, suggested_table, parent_local_id)
            subtasks = staging_df[staging_df["issue_type"] == "Subtask"].copy()
            parsed_rows = []
            pat = re.compile(r'^Add field\s+"([^"]+)"\s+to table\s+"([^"]+)"$', re.IGNORECASE)
            for _, r in subtasks.iterrows():
                m = pat.match((r.get("summary") or "").strip())
                if not m:
                    continue
                parsed_rows.append({
                    "missing_field_name": m.group(1).strip(),
                    "suggested_table":    m.group(2).strip(),
                    "parent_local_id":    str(r.get("parent_local_id") or "").strip()
                })
            fields_for_sql = pd.DataFrame(
                parsed_rows,
                columns=["missing_field_name", "suggested_table", "parent_local_id"]
            )

            # Generate one ALTER per subtask and merge into staging_df.sql_script
            if not fields_for_sql.empty:
                print("\n🛠️ Generating ALTER TABLE statements (one per subtask)...")
                ddl_df = generate_alter_sql_per_field_from_staging(
                    fields_df=fields_for_sql,
                    staging_df=staging_df,
                    schema_text=schema_text
                )
                if not ddl_df.empty:
                    staging_df = staging_df.merge(
                        ddl_df[["local_id", "sql_script"]],
                        on="local_id",
                        how="left"
                    )
                    if "sql_script" in staging_df.columns:
                        staging_df["sql_script"] = staging_df["sql_script"].apply(
                            lambda x: x.strip() if isinstance(x, str) and x.strip() else None
                        )
                else:
                    print("No DDL scripts generated.")
            else:
                print("No standardized 'Add field ...' subtasks found; skipping DDL generation.")

            # Identify tabular reports, then generate SELECT SQL for them
            print("\n📌 Identifying tabular report tasks...")
            staging_df = identify_tabular_reports(staging_df)

            print("\n📊 Generating SELECT SQL queries for tabular reports...")
            staging_df = generate_select_sql_for_reports(
                staging_df=staging_df,
                schema_text=schema_text,
                fields_df=fields_for_sql  # enrich queries with mapped fields if available
            )

    # Priority prediction after all content is in place
    print("\n🔺 Predicting priority for each item...")
    staging_df = add_priority_predictions(staging_df)  # adds 'priority'

    # ===== 3) FINAL DATAFRAME APPROVAL LOOP (values-only edits) =====
    final_ok = False
    while not final_ok:
        final_ok, fb = ask_final_approval(staging_df)
        if final_ok:
            break

        if fb:
            print("\nApplying your feedback. Please wait...")
            staging_df = apply_feedback_to_dataframe_values(
                staging_df,
                feedback=fb,
                allowed_columns=["label", "assignee_group", "priority", "summary", "description"],
                extra_context_columns=["label", "assignee_group", "priority"],
                model="gpt-4o-mini",
                temperature=0.0
            )
        else:
            print("No feedback provided. Please approve or provide value-level feedback.")

    # ===== 4) CREATE JIRA ISSUES =====
    print("\n📝 Creating Jira issues...")
    id_map = create_jira_from_df(staging_df, attach_sql=bool(include_sql), sql_dir="sql_scripts")

    return None

---
### Run the pipeline ###

In [38]:
# Document Path
doc_path = "/Users/georgiospefanis/Desktop/Project_Documents/Technical_Document.docx"

In [39]:
# Load Database Schema
schema_text = load_schema_txt("/Users/georgiospefanis/Desktop/Project_Documents/Database_Schema.txt")

In [ ]:
# 1) Full run WITH SQL (requires schema_text and no Assignee Groups)
run_pipeline(doc_path, schema_text, include_sql=True, include_assignee_group=False)

In [ ]:
# 2) Fast run WITHOUT SQL (no schema needed)
#run_pipeline(doc_path, schema_text=None, include_sql=False)